# 4 or more ML Classifiers - Submission

Will basically copy over first_model notebook things, and then make new models sections. Thinking of doing Logistic Regression, XGBoost, and SVM. Open to other models.